In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_feats_0.2.csv')
#df= pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
#df =pd.read_csv('../../data/more_features/more_feats_new_labels_0.1_noisy_0.6.csv')
df = pd.read_csv('../../data/more_features//more_feats_correlated_noisy_8.csv')
#df = utils.balance_dataset(df, 8000)
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.317680,-1.000000,2.088832,-1.000000,-1.000000,90.986272,113.064122,3.731666,1,1.276701,81.145964,78.146183,29.513433,26.029977,105.808765,33.953040,-1.000000,Aplastic anemia
1,10.109186,-1.000000,-1.000000,-1.000000,-1.000000,104.941225,107.239455,2.889957,0,1.191488,86.444945,103.728730,32.453257,20.009164,90.105559,30.327558,-1.000000,Unspecified anemia
2,6.289434,-1.000000,4.156000,-1.000000,-1.000000,86.013564,-1.000000,2.193642,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,18.868301,-1.000000,Hemolytic anemia
3,7.077683,97.195746,0.261727,0.142422,263.478460,94.058079,102.335651,2.257440,1,-1.000000,114.702341,64.345736,20.102258,3.865156,-1.000000,21.233048,38.840234,No anemia
4,9.717874,101.889169,-1.000000,3.937127,475.993105,78.258930,-1.000000,3.725277,0,1.367858,-1.000000,-1.000000,38.308559,-1.000000,-1.000000,29.153622,-1.000000,No anemia


In [4]:
df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [5]:
utils.get_dt_performance(df)

(0.6207142857142857,
 0.6122228660331646,
 0.7775530397555184,
 datetime.timedelta(microseconds=4547))

In [6]:
df.label.value_counts()

No anemia                               16000
Anemia of chronic disease                8801
Iron deficiency anemia                   8362
Unspecified anemia                       8134
Aplastic anemia                          8097
Vitamin B12/Folate deficiency anemia     8085
Hemolytic anemia                         8068
Inconclusive diagnosis                   4453
Name: label, dtype: int64

In [7]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [8]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [9]:
len(action_list)

25

In [10]:
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.317680,-1.000000,2.088832,-1.000000,-1.000000,90.986272,113.064122,3.731666,1,1.276701,81.145964,78.146183,29.513433,26.029977,105.808765,33.953040,-1.000000,6
1,10.109186,-1.000000,-1.000000,-1.000000,-1.000000,104.941225,107.239455,2.889957,0,1.191488,86.444945,103.728730,32.453257,20.009164,90.105559,30.327558,-1.000000,2
2,6.289434,-1.000000,4.156000,-1.000000,-1.000000,86.013564,-1.000000,2.193642,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,18.868301,-1.000000,5
3,7.077683,97.195746,0.261727,0.142422,263.478460,94.058079,102.335651,2.257440,1,-1.000000,114.702341,64.345736,20.102258,3.865156,-1.000000,21.233048,38.840234,0
4,9.717874,101.889169,-1.000000,3.937127,475.993105,78.258930,-1.000000,3.725277,0,1.367858,-1.000000,-1.000000,38.308559,-1.000000,-1.000000,29.153622,-1.000000,0


#### Training 

In [11]:
# %%time
# timesteps = int(2e6)
# dqn_model = utils.stable_dqn3(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn3_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
#for steps in [int(6e6), int(6.5e6), int(7e6), int(7.5e6), int(8e6), int(8.5e6), int(9e6)]:
for steps in [int(16e6), int(17e6), int(18e6), int(19e6)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_8_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.91     |
|    ep_rew_mean      | -0.78    |
|    exploration_rate | 0.832    |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 756      |
|    time_elapsed     | 374      |
|    total_timesteps  | 283150   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 7.76     |
|    n_updates        | 58287    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.29     |
|    ep_rew_mean      | -0.78    |
|    exploration_rate | 0.641    |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 729      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.91     |
|    ep_rew_mean      | -0.22    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.43     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 527      |
|    time_elapsed     | 12521    |
|    total_timesteps  | 6604364  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.195    |
|    n_updates        | 1638590  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.83     |
|    ep_rew_mean      | -0.12    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.44     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 521      |
|    time_elapsed     | 13465    |
|    total_timesteps  | 7026073  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.57     |
|    ep_rew_mean      | 0.08     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.57     |
| time/               |          |
|    episodes         | 3100000  |
|    fps              | 475      |
|    time_elapsed     | 27389    |
|    total_timesteps  | 13026991 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.13     |
|    n_updates        | 3244247  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.03     |
|    ep_rew_mean      | 0.04     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.56     |
| time/               |          |
|    episodes         | 3200000  |
|    fps              | 473      |
|    time_elapsed     | 28418    |
|    total_timesteps  | 13458699 |
| train/              |          |
|    learning_rate  

#### Testing

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_16000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
test_df[test_df.y_pred==4]

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/test_df3_9000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/success_df3_9000000.csv', index=False)

#### Confusion matrix and classification report

In [ ]:
test_df = pd.read_csv('../../test_dfs/many_features/0.1/test_df3_6500000.csv')
test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])